# Exploration of Forecasting Algorithms on Rossmann Store Sales Dataset


Our goal is to find out if can we can forecast sales of Rossmann stores for 6 weeks in advance.

This is based on Kaggle's Rossmann Store Sales competition. The idea behind the notebook is to maintain
basic code snippets of:

1. Data Handling
2. Data Cleaning
3. Exploring Data through Visualizations
4. Data Analysis
5. Forecasting Techniques

In [3]:
#Import Packages
import pandas as pd
import numpy as np
import sklearn.preprocessing as preprocessing
import sklearn.ensemble as es
import pandas_profiling
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn import tree
from sklearn import neighbors, datasets
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
import seaborn as sns
from sklearn.tree import export_graphviz
import subprocess
from sklearn import linear_model
from pandas import Series, DataFrame
from sklearn.neighbors import KNeighborsRegressor
from math import sqrt
from sklearn.preprocessing import Normalizer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn import datasets
from sklearn.datasets import load_iris
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score
from sklearn.tree import DecisionTreeClassifier
import math as math
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import explained_variance_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree
import math as math
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import pandas as pd
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

In [4]:
#Import Data
train_input = pd.read_csv('train.csv', parse_dates=[2])
test_input  = pd.read_csv('test.csv',parse_dates=[3])
store = pd.read_csv('store.csv')

C:\Users\newton\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
#Check dimensions of the input dataset
print('test_input ', test_input.shape) #test  (41088, 8)
print('train_input ',train_input.shape) #train  (1017209, 9)

test_input  (41088, 8)
train_input  (1017209, 9)


In [6]:
#Cleaning StateHoliday Column
train_input['StateHoliday']=train_input['StateHoliday'].str.replace('0','0').fillna('0')

In [7]:
#Checking for Nulls in the dataset
print('Store variable null counts:\n',store.isnull().sum()) 

#There exists a lot of nulls in the ompetitionOpenSinceMonth,Promo2Since which we will have to treat.

Store variable null counts:
 Store                          0
StoreType                      0
Assortment                     0
CompetitionDistance            3
CompetitionOpenSinceMonth    354
CompetitionOpenSinceYear     354
Promo2                         0
Promo2SinceWeek              544
Promo2SinceYear              544
PromoInterval                544
dtype: int64


In [8]:
#Replacing the nulls with zero
store.fillna(0,inplace = True)

In [9]:
#Merge with store level data

train_df = pd.merge(train_input,store,on='Store')
test_df = pd.merge(test_input,store,on='Store')

#Check dimensions of the merged dataset
print('test ', test_df.shape) #test  (41088, 17)
print('train ',train_df.shape) #train  (1017209, 18)

test  (41088, 17)
train  (1017209, 18)


In [9]:
#Checking for Nulls in the combined dataset

print('Test variable null counts:\n',test_df.isnull().sum()) 
print('\nTrain variable null counts:\n', train_df.isnull().sum()) 

#There exists nulls in the open column which we will have to treat

Test variable null counts:
 Id                            0
Store                         0
DayOfWeek                     0
Date                          0
Open                         11
Promo                         0
StateHoliday                  0
SchoolHoliday                 0
StoreType                     0
Assortment                    0
CompetitionDistance           0
CompetitionOpenSinceMonth     0
CompetitionOpenSinceYear      0
Promo2                        0
Promo2SinceWeek               0
Promo2SinceYear               0
PromoInterval                 0
dtype: int64

Train variable null counts:
 Store                        0
DayOfWeek                    0
Date                         0
Sales                        0
Customers                    0
Open                         0
Promo                        0
StateHoliday                 0
SchoolHoliday                0
StoreType                    0
Assortment                   0
CompetitionDistance          0
CompetitionOp

In [10]:
#Dropping columns that are not required as predictor variables
train_df_dropped = train_df.copy().drop(['Customers'],axis=1).fillna(1)
test_df_dropped = test_df.copy().drop(["Id"],axis=1).fillna(1)

In [11]:
print('test ', test_df_dropped.shape) #test  (41088, 16)
print('train ',train_df_dropped.shape) #train  (1017209, 17)


test  (41088, 16)
train  (1017209, 17)


In [12]:
# Subset for open days and with store sales > 0
train_df_dropped = train_df_dropped[(train_df_dropped.Open != 0)&(train_df_dropped.Sales >0)]

In [13]:
# The below function returns the week of the month for the specified date
import math

def week_of_month(dt):
    """.
    """
    first_day = dt.replace(day=1)
    dom = dt.day
    adjusted_dom = dom + first_day.weekday()
    return int(math.ceil(adjusted_dom/7.0))


In [14]:
# Creating date variables for train (Month, Year, Day, Week, WeekOfMonth)

train_df_dropped['Month'] = train_df_dropped.Date.dt.month_name()
train_df_dropped['Year'] = train_df_dropped.Date.dt.year
train_df_dropped['Day'] = train_df_dropped.Date.dt.day_name()
train_df_dropped['Week'] = train_df_dropped.Date.dt.week
train_df_dropped['WeekOfMonth'] = train_df_dropped.Date.apply(week_of_month)


In [15]:
# Creating date variables for test (Month, Year, Day, Week, WeekOfMonth)

test_df_dropped['Month'] = test_df_dropped.Date.dt.month_name()
test_df_dropped['Year'] = test_df_dropped.Date.dt.year
test_df_dropped['Day'] = test_df_dropped.Date.dt.day_name()
test_df_dropped['Week'] = test_df_dropped.Date.dt.week
test_df_dropped['WeekOfMonth'] = test_df_dropped.Date.apply(week_of_month)


In [16]:
# Calculate promo2 open time in months
test_df_dropped['PromoOpen'] = 12 * (test_df_dropped['Year'] - test_df_dropped.Promo2SinceYear) + \
    (test_df_dropped.Week - test_df_dropped.Promo2SinceWeek) / 4.0
test_df_dropped['PromoOpen'] = test_df_dropped['PromoOpen'].apply(lambda x: x if x > 0 else 0)

train_df_dropped['PromoOpen'] = 12 * (train_df_dropped['Year'] - train_df_dropped.Promo2SinceYear) + \
    (train_df_dropped.Week - train_df_dropped.Promo2SinceWeek) / 4.0
train_df_dropped['PromoOpen'] = train_df_dropped['PromoOpen'].apply(lambda x: x if x > 0 else 0)



In [17]:
# Function to calculate the monthly avg sales historically

#Monthly Average
def monthAvg(month):
    return train_df_dropped.groupby(['Month'])['Sales'].mean()[month]

#Daily Average
def dayAvg(day,week):
    return train_df_dropped.groupby(['Week', 'Day'])['Sales'].mean()[week,day]

#Week Avg
def weekAvg(week):
    return train_df_dropped.groupby(['Week'])['Sales'].mean()[week]



In [17]:
train_df_dropped['month_sales_avg'] = train_df_dropped.apply(lambda x: monthAvg(x['Month']),axis = 1)
train_df_dropped['day_sales_avg'] = train_df_dropped.apply(lambda x: dayAvg(x['Day'],x['Week']),axis = 1)
train_df_dropped['week_sales_avg'] = train_df_dropped.apply(lambda x: weekAvg(x['Week']),axis = 1)

In [18]:
test_df_dropped['month_sales_avg'] = test_df_dropped.apply(lambda x: monthAvg(x['Month']),axis = 1)
test_df_dropped['day_sales_avg'] = test_df_dropped.apply(lambda x: dayAvg(x['Day'],x['Week']),axis = 1)
test_df_dropped['week_sales_avg'] = test_df_dropped.apply(lambda x: weekAvg(x['Week']),axis = 1)

In [19]:
# Function to calculate the monthly avg sales historically based on store type

#Monthly Average
def monthAvgStore(StoreType, month):
    return train_df_dropped.groupby(['StoreType','Month'])['Sales'].mean()[StoreType,month]

#Daily Average
def dayAvgStore(StoreType,day,week):
    return train_df_dropped.groupby(['StoreType','Week', 'Day'])['Sales'].mean()[StoreType,week,day]

#Week Avg
def weekAvgStore(StoreType,week):
    return train_df_dropped.groupby(['StoreType','Week'])['Sales'].mean()[StoreType,week]



In [26]:
train_df_dropped['month_sales_avg_store'] = train_df_dropped.apply(lambda x: monthAvgStore(x['StoreType'],x['Month']),axis = 1)
train_df_dropped['day_sales_avg_store'] = train_df_dropped.apply(lambda x: dayAvgStore(x['StoreType'],x['Day'],x['Week']),axis = 1)
train_df_dropped['week_sales_avg_store'] = train_df_dropped.apply(lambda x: weekAvgStore(x['StoreType'],x['Week']),axis = 1)

In [29]:
test_df_dropped['month_sales_avg_store'] = test_df_dropped.apply(lambda x: monthAvgStore(x['StoreType'],x['Month']),axis = 1)
test_df_dropped['day_sales_avg_store'] = test_df_dropped.apply(lambda x: dayAvgStore(x['StoreType'],x['Day'],x['Week']),axis = 1)
test_df_dropped['week_sales_avg_store'] = test_df_dropped.apply(lambda x: weekAvgStore(x['StoreType'],x['Week']),axis = 1)

In [30]:
# Function to calculate the monthly avg sales historically based on store 

#Monthly Average
def monthAvgSt(Store, month):
    return train_df_dropped.groupby(['Store','Month'])['Sales'].mean()[Store,month]

#Daily Average
def dayAvgSt(Store,day,week):
    return train_df_dropped.groupby(['Store','Week', 'Day'])['Sales'].mean()[Store,week,day]

#Week Avg
def weekAvgSt(Store,week):
    return train_df_dropped.groupby(['Store','Week'])['Sales'].mean()[Store,week]



In [37]:
train_df_dropped['month_sales_avg_st'] = train_df_dropped.apply(lambda x: monthAvgSt(x['Store'],x['Month']),axis = 1)
train_df_dropped['day_sales_avg_st'] = train_df_dropped.apply(lambda x: dayAvgSt(x['Store'],x['Day'],x['Week']),axis = 1)
train_df_dropped['week_sales_avg_st'] = train_df_dropped.apply(lambda x: weekAvgSt(x['Store'],x['Week']),axis = 1)

In [31]:
test_df_dropped['month_sales_avg_st'] = test_df_dropped.apply(lambda x: monthAvgSt(x['Store'],x['Month']),axis = 1)
test_df_dropped['day_sales_avg_st'] = test_df_dropped.apply(lambda x: dayAvgSt(x['Store'],x['Day'],x['Week']),axis = 1)
test_df_dropped['week_sales_avg_st'] = test_df_dropped.apply(lambda x: weekAvgSt(x['Store'],x['Week']),axis = 1)

In [38]:
# For hold-out evaluation,  split the last 6 weeks of train data as hold-out set

train_sorted = train_df_dropped.sort_values(['Date'],ascending = False)
train_df = train_sorted.copy()

split_index = 6*7*1115
valid_df = train_df[:split_index] 
train_final = train_df[split_index:]


In [39]:
#Drop variables to create the X variables to predict
#We need to drop the store variable as there are too many stores to create a one-hot dummy variable
valid = valid_df.drop(['Sales',"Date",'Store'],axis=1)
train = train_df.drop(['Sales',"Date",'Store'],axis=1)
test = test_df_dropped.drop(["Date",'Store'],axis=1)

In [ ]:
#Check the datatypes of the final columns to check for the categorical columns
train_final.dtypes

In [40]:
train_objs_num = len(train)
valid_objs_num = len(valid)
dataset = pd.concat(objs=[train, valid, test], axis=0)

In [41]:
#Transform non numeric columns
cols_to_transform = [  'DayOfWeek' ,'StoreType','Assortment',  'PromoInterval','StateHoliday',  'Month', 'Day', 'Week','WeekOfMonth']
dataset = pd.get_dummies(dataset ,columns = cols_to_transform )

In [42]:
#Split the combined dataset to train and test
train_final = (dataset[:train_objs_num])
valid_final = (dataset[train_objs_num:valid_objs_num+train_objs_num])
test_final = (dataset[valid_objs_num+train_objs_num:])

In [52]:
#Total Train Dataset for Nested Cross Validation
train_final_total = (dataset[:valid_objs_num+train_objs_num])
train_final_total_sales = pd.concat(objs=[train_df['Sales'], valid_df['Sales']], axis=0)

In [25]:
# Get Y for training and validation data
# Through trial and error we see that we have better performance when we take a log transformation of the predictor variable
y_train =  np.log1p(train_df['Sales'])
y_valid =  np.log1p(valid_df['Sales'])

In [26]:
# define eval metrics - based on the problem submissions are evaluated on the Root Mean Square Percentage Error (RMSPE).
def rmspe(y, yhat):
    return np.sqrt(np.mean((yhat/y-1) ** 2))

In [ ]:
# Model 1 - Decision Tree
# Gridsearch and fitting the model
from sklearn.tree import DecisionTreeRegressor
params = {'max_depth': np.arange(3, 10),
                 'min_samples_split': np.linspace(0.01, 0.3, 30, endpoint=True), 
                 'min_samples_leaf': np.linspace(0.01, 0.15, 15, endpoint=True)}
reg = DecisionTreeRegressor()
grid = GridSearchCV(reg, params, cv = 10, refit = True)
grid.fit(train_final,train_df['Sales'])

In [99]:
y_pred = grid.predict(valid_final)
rmspe(valid_df['Sales'],y_pred)

# 0.22786647671385682

0.22786647671385682

In [ ]:
# First submission attempt
y_pred = grid.predict(test_final)
submission = pd.DataFrame({
        "Id": test_df['Id'],
        "Sales": y_pred
    })
submission.to_csv('MSBA.Session1.Team10_dt.csv', index=False)

In [ ]:
# Nested Cross validation - Decision Tree
from sklearn.tree import DecisionTreeRegressor
NUM_TRIALS = 5
p_grid_DT = {'max_depth': np.arange(3, 10),
                 'min_samples_split': np.linspace(0.01, 0.3, 30, endpoint=True), 
                 'min_samples_leaf': np.linspace(0.01, 0.15, 15, endpoint=True)}

dtc = DecisionTreeRegressor()

non_nested_scores_dt = np.zeros(NUM_TRIALS)
nested_scores_dt = np.zeros(NUM_TRIALS)

# Loop for each trial
for i in range(NUM_TRIALS):
    print('\n Iteration number :',i)
    inner_cv = KFold(n_splits=4, shuffle=True, random_state=i)
    outer_cv = KFold(n_splits=4, shuffle=True, random_state=i)

    ## Decision Tree
    # Non_nested parameter search and scoring
    clf = GridSearchCV(estimator=dtc, param_grid = p_grid_DT, cv=inner_cv,scoring = 'neg_mean_squared_error')
    clf.fit(X=train_final_total, y=train_final_total_sales.values)
    non_nested_scores_dt[i] = clf.best_score_

    # Nested CV with parameter optimization
    nested_score_dt = cross_val_score(clf, X=train_final_total, y=train_final_total_sales.values, cv=outer_cv)
    nested_scores_dt[i] = nested_score_dt.mean()



In [ ]:
# Model 2 - KNN
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import StratifiedKFold

scaler = MinMaxScaler(feature_range=(0,1))
X_train = scaler.fit_transform(train_final)
X_valid = scaler.fit_transform(valid_final)
X_test = scaler.fit_transform(test_final)

# build regression pipeline
pipeline = Pipeline([('normalize', Normalizer()),
                     ('kbest', SelectKBest(f_classif)),
                     ('regressor', KNeighborsRegressor())])
knns = KNeighborsRegressor()
# try knn__n_neighbors from 1 to 20, and feature count from 1 to len(features)
parameters = {'kbest__k':  list(range(1, X_train.shape[1]+1)),
              'regressor__n_neighbors': list(range(1,11))}
grid = GridSearchCV(pipeline, parameters, cv=10, scoring="neg_mean_squared_error")
grid.fit(X_train, train_df['Sales'])


In [ ]:
y_pred = grid.predict(X_valid)
rmspe(valid_df['Sales'],y_pred)

# 0.13662021474466066

In [ ]:
# Second submission attempt
y_pred = rf.predict(X_test)
submission = pd.DataFrame({
        "Id": test_df['Id'],
        "Sales": y_pred
    })
submission.to_csv('MSBA.Session1.Team10_knn.csv', index=False)

In [ ]:
# Model 3 - Random Forest
import sklearn.ensemble as es
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
rf = es.RandomForestRegressor()
rf = GridSearchCV(estimator = rf, param_grid = grid, cv = 10, verbose=10, n_jobs = -1)
rf.fit(train_final,train_df['Sales'])

In [83]:
y_pred = rf.predict(valid_final)
rmspe(valid_df['Sales'],y_pred)

# 0.13962021474466066

0.13662021474466066

In [81]:
# Third submission attempt
y_pred = rf.predict(test_final)
submission = pd.DataFrame({
        "Id": test_df['Id'],
        "Sales": y_pred
    })
submission.to_csv('MSBA.Session1.Team10_rf.csv', index=False)

In [ ]:
# Model 4 - Neural Network 
# Define base model
def base_model():
    model = Sequential()
    model.add(Dense(200, input_dim=n_cols, kernel_initializer='normal',activation='relu'))
    model.add(Dense(100, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer='adamax')
    return model

model = KerasRegressor(build_fn=base_model, verbose=0)

## Tuning Hyper -Parameter
# Optimizing for Batch Size and Epochs
batch_size = [5, 20, 40, 100]
epochs = [10, 50, 100,300]

#get number of columns in training data
n_cols = train_final.shape[1]

param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid,
                    scoring='neg_mean_squared_error',n_jobs=1,cv=10)
grid_result = grid.fit(train_final,train_df['Sales'])

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


# Optimizing for Size of first and second layers
def base_model(l1=200,l2=100):
    model = Sequential()
    model.add(Dense(l1, input_dim=n_cols, kernel_initializer='normal' ,activation='relu'))
    model.add(Dense(l2, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer='Adamax')
    return model

model = KerasRegressor(build_fn=base_model, verbose=0, epochs=10, batch_size=5)
l1 = [60, 200, 500, 600, 1000]
l2 = [100, 250 ,400, 500, 600, 700]
param_grid = dict(l1=l1, l2=l2)
grid = GridSearchCV(estimator=model, param_grid=param_grid,scoring='neg_mean_squared_error')
grid_result = grid.fit(train_final,train_df['Sales'])

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Final model on best hyper parameter setting
def final_model():
    model = Sequential()
    model.add(Dense(1000, input_dim=n_cols, kernel_initializer='normal' ,activation='relu'))
    model.add(Dense(600, kernel_initializer='normal',activation='relu',))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer='Adamax')
    return model

reg = KerasRegressor(build_fn=final_model, epochs=300, batch_size=40,verbose=1,validation_split=0.2)
kfold = KFold(n_splits=5, random_state=1)
results = np.sqrt(-1*cross_val_score(reg, train_final,train_df['Sales'],scoring= "neg_mean_squared_error", cv=kfold))
print("Training RMSE mean and std from CV: {} {}".format(results.mean(),results.std()))

# Prediction Evalution on testing data
reg.fit(train_final,train_df['Sales'])
y_pred = reg.predict(train_final)

y_pred = reg.predict(valid_final)
rmspe(valid_df['Sales'],y_pred)

y_pred = reg.predict(test_final)



In [ ]:
# Fourth submission attempt
y_pred = reg.predict(test_final)
submission = pd.DataFrame({
        "Id": test_df['Id'],
        "Sales": y_pred
    })
submission.to_csv('MSBA.Session1.Team10_nn.csv', index=False)

In [ ]:
#Feature selection using Random Forest
clf = RandomForestRegressor(n_estimators=100, random_state=1)
clf.fit(train_final,train_df['Sales'])
feature_importance = pd.DataFrame(np.round(clf.feature_importances_,3),
                        index = X_train.columns,
                        columns=['importance']).sort_values('importance',
                        ascending = False)

#Features are ranked based on how they reduce imprurity and create pure nodes.
#Based on the ranking, we decide to pick up the top 10 features
#which have importance greater than 0.009.

#Based on the values, we take the top 10 features and compare how 
#performance differs.
top_features = feature_importance.nlargest(10,"importance")
top_features .T.plot(kind = "bar")

# Selecting top features for training
scaler = MinMaxScaler()
x_train_scaled_fs = scaler.fit_transform(X_train.filter(items = top_features.index))
x_valid_scaled_fs = scaler.transform(X_test.filter(items = top_features.index))
x_test_scaled_fs = scaler.transform(X_test.filter(items = top_features.index))

In [ ]:
# Model 5 - SVR

#Grid Search SVM Regression
parameters = {'kernel':['linear','rbf','poly'], 'C':[0.1, 1, 10, 100, 1000],
              'gamma': [0.1, 1, 10],'degree': [0, 1, 2]}
clf = GridSearchCV(SVR(), parameters,cv= 10, verbose = 10,scoring='neg_mean_squared_error',refit=True,n_jobs=-1)
clf.fit(x_train_scaled_fs,train_df['Sales']) 
clf.best_params_
y_pred = clf.predict(x_valid_scaled_fs)
rmspe(valid_df['Sales'],y_pred)
# Five submission attempt
y_pred = clf.predict(x_test_scaled_fs)
submission = pd.DataFrame({
        "Id": test_df['Id'],
        "Sales": y_pred
    })
submission.to_csv('MSBA.Session1.Team10_svr.csv', index=False)

In [ ]:
# Model 6 - GBM
# Setting the hyperparameters by cross-validation
gbm_parameters = [{'n_estimators': [100,500,1000],
                    'max_depth': [6,10,14],
                    'max_features': [4,8,12],
                    'min_samples_leaf': [5,10,15]}]

reg = GridSearchCV(GradientBoostingRegressor(), gbm_parameters, cv=10, scoring = 'neg_mean_squared_error')
reg.fit(train_final,train_df['Sales'])

print("Best parameters set found on development set:")
print(reg.best_params_)

print("Best score found on development set:")
print(reg.best_score_)

y_pred = reg.predict(valid_final)
rmspe(valid_df['Sales'],y_pred)


In [ ]:
# Model 7 - Adaboost
# Setting the hyperparameters by cross-validation
ab_parameters = [{'n_estimators': [10,50,100,200],
                    'learning_rate': [0.01,0.1,0.3]}]

reg = GridSearchCV(AdaBoostRegressor(), ab_parameters, cv=10, scoring='neg_mean_squared_error')
reg.fit(X_train, y_train)

print("Best parameters set found on development set:")
print(reg.best_params_)

print("Best score found on development set:")
print(reg.best_score_)


# Final Adaboost model accuracy using the hyper-parameters obtained from grid search
y_pred = reg.predict(X_test)
reg.score(X_test,y_test) 



In [ ]:
# NESTED CROSS VALIDATION

NUM_TRIALS = 5

gbm = GradientBoostingRegressor()
ab = AdaBoostRegressor()
rf = RandomForestRegressor()

# Arrays to store scores
non_nested_scores_gbm = np.zeros(NUM_TRIALS)
nested_scores_gbm = np.zeros(NUM_TRIALS)

non_nested_scores_ab = np.zeros(NUM_TRIALS)
nested_scores_ab = np.zeros(NUM_TRIALS)

non_nested_scores_rf = np.zeros(NUM_TRIALS)
nested_scores_rf = np.zeros(NUM_TRIALS)

# Loop for each trial
for i in range(NUM_TRIALS):
    print('\n Iteration number :',i)
    # Choose cross-validation techniques for the inner and outer loops,
    # independently of the dataset.
    # E.g "LabelKFold", "LeaveOneOut", "LeaveOneLabelOut", etc.
    inner_cv = KFold(n_splits=4, shuffle=True, random_state=i)
    outer_cv = KFold(n_splits=4, shuffle=True, random_state=i)
    
    ## GBM
    # Non_nested parameter search and scoring
    clf = GridSearchCV(estimator=gbm, param_grid=gbm_parameters, cv=inner_cv)
    clf.fit(X=train_final_total, y=train_final_total_sales.values)
    non_nested_scores_gbm[i] = clf.best_score_
    
    # Nested CV with parameter optimization
    nested_score_gbm = cross_val_score(clf, X=train_final_total, y=train_final_total_sales.values, cv=outer_cv)
    nested_scores_gbm[i] = nested_score_gbm.mean()
    
    ## Adaboost
    # Non_nested parameter search and scoring
    clf = GridSearchCV(estimator=ab, param_grid = ab_parameters, cv=inner_cv)
    clf.fit(X=train_final_total, y=train_final_total_sales.values)
    non_nested_scores_ab[i] = clf.best_score_

    # Nested CV with parameter optimization
    nested_score_ab = cross_val_score(clf, X=train_final_total, y=train_final_total_sales.values, cv=outer_cv)
    nested_scores_ab[i] = nested_score_ab.mean()

    ## Random Forest
    # Non_nested parameter search and scoring
    clf = GridSearchCV(estimator=rf, param_grid = rf_parameters, cv=inner_cv)
    clf.fit(X=train_final_total, y=train_final_total_sales.values)
    non_nested_scores_rf[i] = clf.best_score_

    # Nested CV with parameter optimization
    nested_score_rf = cross_val_score(clf, X=train_final_total, y=train_final_total_sales.values, cv=outer_cv)
    nested_scores_rf[i] = nested_score_rf.mean()
    
score_difference_gbm = non_nested_scores_gbm - nested_scores_gbm
score_difference_ab = non_nested_scores_ab - nested_scores_ab
score_difference_rf = non_nested_scores_rf - nested_scores_rf



In [ ]:
# Model 6 - XGBoost

# A parameter grid for XGBoost
xgb_model = XGBRegressor()
parameters = {'objective':['reg:linear'],
              'learning_rate': [0.02,0.03,0.04,0.05], #so called `eta` value
              'max_depth': [6,7,8,9,10,11,12,13],
              'subsample': [0.7,0.8,0.09,],
              'colsample_bytree': [0.7,0.8,0.9]}

clf = GridSearchCV(xgb_model, parameters, n_jobs=5, cv=10, verbose=2, refit=True)
clf.fit(train_final,train_df['Sales']) 
clf.best_params_
y_pred = clf.predict(valid_final)
rmspe(valid_df['Sales'],y_pred)


In [ ]:
# NESTED CROSS VALIDATION

NUM_TRIALS = 5

xgb_model = XGBRegressor()

# Arrays to store scores

non_nested_scores_xg = np.zeros(NUM_TRIALS)
nested_scores_xg = np.zeros(NUM_TRIALS)

# Loop for each trial
for i in range(NUM_TRIALS):
    print('\n Iteration number :',i)
    # Choose cross-validation techniques for the inner and outer loops,
    # independently of the dataset.
    # E.g "LabelKFold", "LeaveOneOut", "LeaveOneLabelOut", etc.
    inner_cv = KFold(n_splits=4, shuffle=True, random_state=i)
    outer_cv = KFold(n_splits=4, shuffle=True, random_state=i)
    
    ## XGB
    # Non_nested parameter search and scoring
    clf = GridSearchCV(estimator=xg_model, param_grid=parameters, cv=inner_cv)
    clf.fit(X=train_final_total, y=train_final_total_sales.values)
    non_nested_scores_xg[i] = clf.best_score_
    
    # Nested CV with parameter optimization
    nested_score_xg = cross_val_score(clf, X=train_final_total, y=train_final_total_sales.values, cv=outer_cv)
    nested_scores_xg[i] = nested_score_xg.mean()
    
score_difference_xg = non_nested_scores_xg - nested_scores_xg


In [ ]:
# For XGboost since we are using DMatrix as input we will need to do a custom loss function to ensure that we can calculate RMSPE
def rmspe_xg(yhat, y):
    y = np.expm1(y.get_label())
    yhat = np.expm1(yhat)
    return "rmspe", rmspe(y,yhat)

In [81]:
# XGboost converting to DMatrix and using num_boost_round on the final selected parameters

import xgboost as xgb

params = {"objective": "reg:linear",  
          "eta": 0.03,  
          "max_depth": 10,
          "subsample": 0.9,
          "colsample_bytree": 0.7
          }
num_boost_round = 4000

dtrain = xgb.DMatrix(train_final, y_train)
dvalid = xgb.DMatrix(valid_final, y_valid)
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
xg_model = xgb.train(params, dtrain, num_boost_round, evals=watchlist, early_stopping_rounds= 100, feval=rmspe_xg, verbose_eval=True)

#Lowest RMSE across eval is 0.063044 

[0]	train-rmse:8.01821	eval-rmse:8.03995	train-rmspe:inf	eval-rmspe:0.999814
Multiple eval metrics have been passed: 'eval-rmspe' will be used for early stopping.

Will train until eval-rmspe hasn't improved in 100 rounds.


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


[1]	train-rmse:7.77813	eval-rmse:7.80019	train-rmspe:inf	eval-rmspe:0.999719
[2]	train-rmse:7.54526	eval-rmse:7.56767	train-rmspe:inf	eval-rmspe:0.999602
[3]	train-rmse:7.3194	eval-rmse:7.34203	train-rmspe:inf	eval-rmspe:0.99946
[4]	train-rmse:7.10031	eval-rmse:7.12322	train-rmspe:inf	eval-rmspe:0.999289
[5]	train-rmse:6.88788	eval-rmse:6.91073	train-rmspe:inf	eval-rmspe:0.999082
[6]	train-rmse:6.6818	eval-rmse:6.70481	train-rmspe:inf	eval-rmspe:0.998835
[7]	train-rmse:6.48194	eval-rmse:6.50486	train-rmspe:inf	eval-rmspe:0.998541
[8]	train-rmse:6.28811	eval-rmse:6.31095	train-rmspe:inf	eval-rmspe:0.998195
[9]	train-rmse:6.10004	eval-rmse:6.12291	train-rmspe:inf	eval-rmspe:0.99779
[10]	train-rmse:5.91769	eval-rmse:5.94054	train-rmspe:inf	eval-rmspe:0.997316
[11]	train-rmse:5.74075	eval-rmse:5.76378	train-rmspe:inf	eval-rmspe:0.996768
[12]	train-rmse:5.56914	eval-rmse:5.59209	train-rmspe:inf	eval-rmspe:0.996136
[13]	train-rmse:5.40269	eval-rmse:5.42542	train-rmspe:inf	eval-rmspe:0.995409

[106]	train-rmse:0.409936	eval-rmse:0.412522	train-rmspe:inf	eval-rmspe:0.322006
[107]	train-rmse:0.402106	eval-rmse:0.404385	train-rmspe:inf	eval-rmspe:0.317089
[108]	train-rmse:0.394615	eval-rmse:0.396558	train-rmspe:inf	eval-rmspe:0.312352
[109]	train-rmse:0.387544	eval-rmse:0.389022	train-rmspe:inf	eval-rmspe:0.307793
[110]	train-rmse:0.380804	eval-rmse:0.3819	train-rmspe:inf	eval-rmspe:0.303473
[111]	train-rmse:0.37448	eval-rmse:0.375212	train-rmspe:inf	eval-rmspe:0.299436
[112]	train-rmse:0.368033	eval-rmse:0.368471	train-rmspe:inf	eval-rmspe:0.295403
[113]	train-rmse:0.360732	eval-rmse:0.360973	train-rmspe:inf	eval-rmspe:0.290802
[114]	train-rmse:0.353779	eval-rmse:0.353766	train-rmspe:inf	eval-rmspe:0.28639
[115]	train-rmse:0.347678	eval-rmse:0.347463	train-rmspe:inf	eval-rmspe:0.282556
[116]	train-rmse:0.342452	eval-rmse:0.341722	train-rmspe:inf	eval-rmspe:0.279137
[117]	train-rmse:0.336945	eval-rmse:0.335885	train-rmspe:inf	eval-rmspe:0.275649
[118]	train-rmse:0.332129	eval-r

[208]	train-rmse:0.201048	eval-rmse:0.187928	train-rmspe:inf	eval-rmspe:0.195435
[209]	train-rmse:0.200864	eval-rmse:0.187704	train-rmspe:inf	eval-rmspe:0.195341
[210]	train-rmse:0.200233	eval-rmse:0.187115	train-rmspe:inf	eval-rmspe:0.194807
[211]	train-rmse:0.200076	eval-rmse:0.186928	train-rmspe:inf	eval-rmspe:0.194729
[212]	train-rmse:0.199381	eval-rmse:0.18625	train-rmspe:inf	eval-rmspe:0.194093
[213]	train-rmse:0.198945	eval-rmse:0.185793	train-rmspe:inf	eval-rmspe:0.193748
[214]	train-rmse:0.198782	eval-rmse:0.185637	train-rmspe:inf	eval-rmspe:0.193705
[215]	train-rmse:0.19849	eval-rmse:0.18524	train-rmspe:inf	eval-rmspe:0.193448
[216]	train-rmse:0.198272	eval-rmse:0.185053	train-rmspe:inf	eval-rmspe:0.193368
[217]	train-rmse:0.198132	eval-rmse:0.184837	train-rmspe:inf	eval-rmspe:0.193238
[218]	train-rmse:0.197777	eval-rmse:0.184451	train-rmspe:inf	eval-rmspe:0.192994
[219]	train-rmse:0.197518	eval-rmse:0.184195	train-rmspe:inf	eval-rmspe:0.192804
[220]	train-rmse:0.19724	eval-r

...

[3549]	train-rmse:0.070399	eval-rmse:0.064843	train-rmspe:inf	eval-rmspe:0.065305
[3550]	train-rmse:0.070395	eval-rmse:0.064822	train-rmspe:inf	eval-rmspe:0.065289
[3551]	train-rmse:0.070386	eval-rmse:0.064835	train-rmspe:inf	eval-rmspe:0.065278
[3552]	train-rmse:0.070379	eval-rmse:0.064833	train-rmspe:inf	eval-rmspe:0.065276
[3553]	train-rmse:0.070375	eval-rmse:0.06483	train-rmspe:inf	eval-rmspe:0.065273
[3554]	train-rmse:0.070369	eval-rmse:0.064824	train-rmspe:inf	eval-rmspe:0.065266
[3555]	train-rmse:0.070367	eval-rmse:0.064822	train-rmspe:inf	eval-rmspe:0.065264
[3556]	train-rmse:0.070362	eval-rmse:0.064815	train-rmspe:inf	eval-rmspe:0.065257
[3557]	train-rmse:0.070356	eval-rmse:0.064814	train-rmspe:inf	eval-rmspe:0.065255
[3558]	train-rmse:0.070351	eval-rmse:0.064814	train-rmspe:inf	eval-rmspe:0.065255
[3559]	train-rmse:0.070346	eval-rmse:0.064811	train-rmspe:inf	eval-rmspe:0.065252
[3560]	train-rmse:0.070335	eval-rmse:0.064809	train-rmspe:inf	eval-rmspe:0.065248
[3561]	train-rmse

[3650]	train-rmse:0.069729	eval-rmse:0.064367	train-rmspe:inf	eval-rmspe:0.064792
[3651]	train-rmse:0.069719	eval-rmse:0.06435	train-rmspe:inf	eval-rmspe:0.064779
[3652]	train-rmse:0.069709	eval-rmse:0.064334	train-rmspe:inf	eval-rmspe:0.064771
[3653]	train-rmse:0.069703	eval-rmse:0.064334	train-rmspe:inf	eval-rmspe:0.06477
[3654]	train-rmse:0.069698	eval-rmse:0.064317	train-rmspe:inf	eval-rmspe:0.064759
[3655]	train-rmse:0.069696	eval-rmse:0.064292	train-rmspe:inf	eval-rmspe:0.064745
[3656]	train-rmse:0.069693	eval-rmse:0.064284	train-rmspe:inf	eval-rmspe:0.064738
[3657]	train-rmse:0.069688	eval-rmse:0.064276	train-rmspe:inf	eval-rmspe:0.064731
[3658]	train-rmse:0.069677	eval-rmse:0.064261	train-rmspe:inf	eval-rmspe:0.064721
[3659]	train-rmse:0.06967	eval-rmse:0.064252	train-rmspe:inf	eval-rmspe:0.064711
[3660]	train-rmse:0.069665	eval-rmse:0.06425	train-rmspe:inf	eval-rmspe:0.064709
[3661]	train-rmse:0.069662	eval-rmse:0.064248	train-rmspe:inf	eval-rmspe:0.064707
[3662]	train-rmse:0.

[3751]	train-rmse:0.069124	eval-rmse:0.063823	train-rmspe:inf	eval-rmspe:0.064275
[3752]	train-rmse:0.069115	eval-rmse:0.063816	train-rmspe:inf	eval-rmspe:0.064266
[3753]	train-rmse:0.069105	eval-rmse:0.063822	train-rmspe:inf	eval-rmspe:0.064263
[3754]	train-rmse:0.069101	eval-rmse:0.06382	train-rmspe:inf	eval-rmspe:0.064261
[3755]	train-rmse:0.069094	eval-rmse:0.063805	train-rmspe:inf	eval-rmspe:0.06425
[3756]	train-rmse:0.069088	eval-rmse:0.0638	train-rmspe:inf	eval-rmspe:0.064248
[3757]	train-rmse:0.069085	eval-rmse:0.0638	train-rmspe:inf	eval-rmspe:0.064248
[3758]	train-rmse:0.069076	eval-rmse:0.063798	train-rmspe:inf	eval-rmspe:0.064245
[3759]	train-rmse:0.069068	eval-rmse:0.063798	train-rmspe:inf	eval-rmspe:0.064245
[3760]	train-rmse:0.069063	eval-rmse:0.063791	train-rmspe:inf	eval-rmspe:0.064238
[3761]	train-rmse:0.069058	eval-rmse:0.063789	train-rmspe:inf	eval-rmspe:0.064234
[3762]	train-rmse:0.069053	eval-rmse:0.063785	train-rmspe:inf	eval-rmspe:0.064231
[3763]	train-rmse:0.06

[3852]	train-rmse:0.068486	eval-rmse:0.063344	train-rmspe:inf	eval-rmspe:0.063778
[3853]	train-rmse:0.068482	eval-rmse:0.063341	train-rmspe:inf	eval-rmspe:0.063776
[3854]	train-rmse:0.06848	eval-rmse:0.063341	train-rmspe:inf	eval-rmspe:0.063775
[3855]	train-rmse:0.068472	eval-rmse:0.063331	train-rmspe:inf	eval-rmspe:0.063764
[3856]	train-rmse:0.068469	eval-rmse:0.063328	train-rmspe:inf	eval-rmspe:0.063761
[3857]	train-rmse:0.068462	eval-rmse:0.063331	train-rmspe:inf	eval-rmspe:0.063761
[3858]	train-rmse:0.068458	eval-rmse:0.06333	train-rmspe:inf	eval-rmspe:0.06376
[3859]	train-rmse:0.06845	eval-rmse:0.063328	train-rmspe:inf	eval-rmspe:0.063756
[3860]	train-rmse:0.068447	eval-rmse:0.063327	train-rmspe:inf	eval-rmspe:0.063756
[3861]	train-rmse:0.068444	eval-rmse:0.063323	train-rmspe:inf	eval-rmspe:0.063751
[3862]	train-rmse:0.068433	eval-rmse:0.063318	train-rmspe:inf	eval-rmspe:0.063747
[3863]	train-rmse:0.068428	eval-rmse:0.063302	train-rmspe:inf	eval-rmspe:0.063732
[3864]	train-rmse:0.

[3953]	train-rmse:0.067871	eval-rmse:0.062915	train-rmspe:inf	eval-rmspe:0.063279
[3954]	train-rmse:0.067866	eval-rmse:0.062919	train-rmspe:inf	eval-rmspe:0.06328
[3955]	train-rmse:0.067863	eval-rmse:0.062914	train-rmspe:inf	eval-rmspe:0.063275
[3956]	train-rmse:0.067853	eval-rmse:0.062909	train-rmspe:inf	eval-rmspe:0.063268
[3957]	train-rmse:0.067848	eval-rmse:0.062892	train-rmspe:inf	eval-rmspe:0.063259
[3958]	train-rmse:0.067845	eval-rmse:0.062888	train-rmspe:inf	eval-rmspe:0.063257
[3959]	train-rmse:0.067842	eval-rmse:0.062886	train-rmspe:inf	eval-rmspe:0.063256
[3960]	train-rmse:0.067836	eval-rmse:0.062884	train-rmspe:inf	eval-rmspe:0.063253
[3961]	train-rmse:0.067831	eval-rmse:0.06287	train-rmspe:inf	eval-rmspe:0.063238
[3962]	train-rmse:0.067824	eval-rmse:0.062867	train-rmspe:inf	eval-rmspe:0.063232
[3963]	train-rmse:0.067815	eval-rmse:0.062865	train-rmspe:inf	eval-rmspe:0.063231
[3964]	train-rmse:0.06781	eval-rmse:0.062861	train-rmspe:inf	eval-rmspe:0.063228
[3965]	train-rmse:0

In [82]:
x_train_total = pd.concat(objs=[train_final, valid_final], axis=0)
y_train_total = pd.concat(objs=[y_train, y_valid], axis=0)

In [84]:
#Training the model on the entire data
dtrain = xgb.DMatrix(x_train_total, y_train_total)
dtest = xgb.DMatrix(test_final)
params = {"objective": "reg:linear",
          "booster" : "gbtree",   
          "eta": 0.03,   
          "max_depth": 10,  
          "subsample": 0.9, 
          "colsample_bytree": 0.7,        
          }
num_round = 1000
xg_model = xgb.train(params, dtrain, num_round)
# make prediction
preds = xg_model.predict(dtest)

In [87]:
# Eight submission attempt
submission = pd.DataFrame({
        "Id": test_df['Id'],
        "Sales": np.expm1(preds)
    })
submission.to_csv('MSBA.Session1.Team10_XGboost.csv', index=False)